In [61]:
import pygame
from typing import Any, Optional, List
from pathlib import Path
from libbci.inputdevice import  InputDevice#from .ble.charcontroller import CharController
import time

class Joystick_controller:
    def __init__(self):
        self.pygame_joystick = pygame.joystick.Joystick(0)
        self.pygame_joystick.init()
        self.num_axes = self.pygame_joystick.get_numaxes()
        self.num_buttons = self.pygame_joystick.get_numbuttons()
    def get_data(self):
        pygame.event.get()
        x_axis_data = self.pygame_joystick.get_axis(0)
        y_axis_data = self.pygame_joystick.get_axis(1)
        self.data_buffer.append([x_axis_data, y_axis_data])
        return self.data_buffer

    def flush_buffer(self):
        self.data_buffer = []

class Joystick_device(InputDevice):
    def __init__(self):
        pygame.init()
        

        # Set up the game controller
        self.pygame_joystick = pygame.joystick.Joystick(0)
        self.pygame_joystick.init()
        # buffer
        self.data_buffer = []
    
    def get_input(self):
        data = self.get_data()
        return data

In [ ]:
#clocking mechanism
#theres an application loop that calls get input
#samples data from a buffer need to expose data when git inbput is called
#eeg heaset pip install and look at get_raw_data


#joystock model get controler1 x axis get controller 2 y axis
#joystick class that allows us to defien the object

#nexus instument and nexus device uses instrument as souce of truth
#get data and joystic controller should return numpy array 6 by t ,6,256 

In [62]:
pygame.event.get()
[
    joystick.pygame_joystick.get_axis(i)
    for i in range(6)
]

[-0.028472900390625,
 -0.079254150390625,
 -1.0,
 -0.10589599609375,
 0.012908935546875,
 -1.0]

In [63]:
joystick = Joystick()

In [64]:

while True:
    joystick.flush_buffer()
    data = joystick.get_input()
    joystick.flush_buffer()
    print(data)
    time.sleep(1)

[[-0.028472900390625, -0.079254150390625]]


[[-0.028472900390625, -0.079254150390625]]
[[-0.028472900390625, -0.079254150390625]]
[[-0.028472900390625, -0.079254150390625]]
[[-0.028472900390625, -0.079254150390625]]
[[0.1297607421875, -0.211273193359375]]
[[-0.110748291015625, -1.0]]
[[0.999969482421875, -0.181549072265625]]
[[0.401885986328125, 0.82830810546875]]
[[-0.892364501953125, 0.346466064453125]]
[[-0.92083740234375, -0.48187255859375]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]
[[-0.02215576171875, -0.069366455078125]]


KeyboardInterrupt: 